# Building a simple RAG chatbot with LangChain, Hugging Face, FAISS, Amazon SageMaker and Amazon Textract

In [2]:
%%sh
pip install sagemaker langchain amazon-textract-caller amazon-textract-textractor sentence-transformers pypdf pip install faiss-cpu -qU

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [3]:
import boto3, json, sagemaker
from typing import Dict
from langchain import LLMChain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.llms import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

## Deploy LLM on SageMaker

In [5]:
#t5 XL
# Hub Model configuration. https://huggingface.co/models
role = sagemaker.get_execution_role()

hub = {
	#'HF_MODEL_ID':'google/flan-t5-small',
    #'HF_MODEL_ID':'bigscience/bloomz-3b',
    #'HF_MODEL_ID':'google/flan-t5-xl',
    'HF_MODEL_ID':'google-t5/t5-base',
	'SM_NUM_GPUS': json.dumps(1)
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.2xlarge",
	container_startup_health_check_timeout=600,
    endpoint_name="flan-t5base-demo"
  )
  
# send request
predictor.predict({
	"inputs": "Translate to German:  My name is Arthur",
})

-------!

[{'generated_text': 'My name is Arthur. My name is Arthur.'}]

In [6]:
endpoint_name = predictor.endpoint_name
endpoint_name

'flan-t5base-demo'

**Zero Shot example** 1. Ask a question to LLM without providing the context
To better illustrate why we need retrieval-augmented generation (RAG) based approach to solve the question and anwering problem. Let's directly ask the model a question and see how they respond.

In [11]:
question = "Which instances can I use with Managed Spot Training in SageMaker?"

out = predictor.predict({"inputs": question})
out

[{'generated_text': 'Which instances can I use with Managed Spot Training in SageMaker?'}]

In [8]:
# define payload
prompt="""<|prompter|>How can i stay more active during winter? Give me 3 tips.<|endoftext|><|assistant|>"""

# hyperparameters for llm
payload = {
  "inputs": prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.7,
    "temperature": 0.7,
    "top_k": 10,
    "min_length": 200,
    "stop": ["<|endoftext|>"]
  }
}

# send request to endpoint
response = predictor.predict(payload)

# print(response[0]["generated_text"][:-len("<human>:")])
print(response[0]["generated_text"])


|prompter|>|prompter|>|help|>


Step 3. Improve the answer to the same question using prompt engineering with insightful context
To better answer the question well, we provide extra contextual information, combine it with a prompt, and send it to model together with the question. Below is an example.

In [18]:
context = """Managed Spot Training can be used with all instances
supported in Amazon SageMaker. Managed Spot Training is supported
in all AWS Regions where Amazon SageMaker is currently available."""

In [19]:
# Define the prompt template for T5
prompt_template = """summarize: CONTEXT: {context} QUESTION: {question}"""

# Replace placeholders with actual context and question
text_input = prompt_template.format(context=context, question=question)

# Predict the answer using the adjusted prompt
out = predictor.predict({"inputs": text_input})
generated_text = out[0]["generated_text"]

# Print the question and the model's output
print(f"[Question]: {question}\n[Answer]: {generated_text}")


[Question]: Which instances can I use with Managed Spot Training in SageMaker?
[Answer]: Managed Spot Training is supported in all AWS regions where Amazon SageMaker is currently available


In [15]:
#summarize based on context
context = ("Data science is an interdisciplinary field[10] focused on extracting knowledge from typically large data sets and applying the knowledge and insights from that data to solve problems in a wide range of application domains.[11] The field encompasses preparing data for analysis, formulating data science problems, analyzing data, developing data-driven solutions, and presenting findings to inform high-level decisions in a broad range of application domains. As such, it incorporates skills from computer science, statistics, information science, mathematics, data visualization, information visualization, data sonification, data integration, graphic design, complex systems, communication and business.[12][13] Statistician Nathan Yau, drawing on Ben Fry, also links data science to human–computer interaction: users should be able to intuitively control and explore data.[14][15] In 2015, the American Statistical Association identified database management, statistics and machine learning, and distributed and parallel systems as the three emerging foundational professional communities.[16]")

prompt_template = """summarize based on the CONTEXT
given.

CONTEXT:
{context}


Summary:
"""

text_input = prompt_template.replace("{context}", context)

payload = {
    "inputs": text_input,
    "parameters": {
        "min_length": 80,
        "max_length": 512
    }
}

# Make the prediction request
out = predictor.predict(payload)
generated_text = out[0]["generated_text"]
print(f"[Summary]: {generated_text}")

[Summary]: data science is an interdisciplinary field[10] focused on extracting knowledge from typically large data


Let's see if our LLM is capable of following our instructions...

In [20]:
unanswerable_question = "What color is my desk?"

text_input = prompt_template.replace("{context}", context).replace("{question}", unanswerable_question)

out = predictor.predict({"inputs": text_input})
generated_text = out[0]["generated_text"]
print(f"[Input]: {unanswerable_question}\n[Output]: {generated_text}")

[Input]: What color is my desk?
[Output]: Managed Spot Training is supported in all AWS regions where Amazon SageMaker is currently available


In [ ]:
sequence = ("Data science is an interdisciplinary field[10] focused on extracting knowledge from typically large data sets and applying the knowledge and insights from that data to solve problems in a wide range of application domains.[11] The field encompasses preparing data for analysis, formulating data science problems, analyzing data, developing data-driven solutions, and presenting findings to inform high-level decisions in a broad range of application domains. As such, it incorporates skills from computer science, statistics, information science, mathematics, data visualization, information visualization, data sonification, data integration, graphic design, complex systems, communication and business.[12][13] Statistician Nathan Yau, drawing on Ben Fry, also links data science to human–computer interaction: users should be able to intuitively control and explore data.[14][15] In 2015, the American Statistical Association identified database management, statistics and machine learning, and distributed and parallel systems as the three emerging foundational professional communities.[16]")

inputs=tokenizer.encode("sumarize: " +sequence,return_tensors='pt', max_length=512, truncation=True)

output = model.generate(inputs, min_length=80, max_length=100)

summary=tokenizer.decode(output[0])
print(summary)


## Configure LLM in LangChain

In [21]:
model_kwargs = {"max_new_tokens": 1024,
    "top_p": 0.8, "temperature": 0.8,"max_length": 512,"min_length":300 }

In [22]:
#define content handler class
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_data = {
            "inputs": prompt,  # Adjust this field based on the expected input format
            **model_kwargs,
        }
        input_str = json.dumps(input_data)
        return input_str.encode("utf-8")
     
    def transform_output(self, output: 'StreamingBody') -> str:
            response_json = json.loads(output.read().decode("utf-8"))
            print(response_json)
            return response_json[0]["generated_text"]


content_handler = ContentHandler()


In [23]:
import boto3
sm_client = boto3.client("sagemaker-runtime") # needed for AWS credentials

llm = SagemakerEndpoint(
    endpoint_name=endpoint_name,
    model_kwargs=model_kwargs,
    content_handler=content_handler,
    client=sm_client,
)



## Zero-shot example

In [55]:
system_prompt = """Answer the following QUESTION based on the CONTEXT
given.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

prompt = PromptTemplate.from_template(system_prompt + "{context}")

In [56]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [57]:
context ="Solar investments trends in China have increased by 30% each year in the last decade"
question = "What is the investment trend for solar investments in China?"

query = f"question: {question}"

In [58]:
query = f"question: {question}"
print(query)

question: What is the investment trend for solar investments in China?


In [59]:
answer = llm_chain.run({"context": context, "question": query})
print(answer)

[{'generated_text': 'True'}]
True


## RAG example with PDF files

In [29]:
from langchain.document_loaders import AmazonTextractPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
#!pip install pypdf
#!pip install pypdf2

#this is working for single pdf it is created as pypdf can't load s3 object directly
import boto3
import tempfile
from io import BytesIO
from langchain.document_loaders import PyPDFLoader
import os

# Specify your S3 bucket name and item name
bucket_name = "bo-automation"
item_name = "langchain-rag-demo/Coal2022.pdf"

# Create an S3 client
s3 = boto3.client("s3")

# Get the PDF file content from S3
response = s3.get_object(Bucket=bucket_name, Key=item_name)
pdf_content = response["Body"].read()

# Use BytesIO to create a file-like object from the PDF content
pdf_file = BytesIO(pdf_content)

# Save the contents to a temporary file
with tempfile.NamedTemporaryFile(delete=False) as temp_file:
    temp_file.write(pdf_content)
    temp_file_path = temp_file.name

# Create a PyPDFLoader instance and load the PDF document
loader = PyPDFLoader(temp_file_path)
docs = loader.load()
print(len(docs))

# Now you can work with the 'document' object, which represents the PDF content
# For example, you can access the pages: document.pages

# Optionally, delete the temporary file
os.remove(temp_file_path)

In [30]:
#working fine to load multiple files
import boto3
import tempfile
from io import BytesIO
from langchain.document_loaders import PyPDFLoader
import os

# Initialize Boto3 S3 client
s3 = boto3.client('s3')
bucket_name = "bo-automation1"
prefix = "langchain-rag-demo/"

# List objects within the specified directory
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# Initialize a list to hold all documents
all_docs = []

# Iterate over each file and load its contents
for obj in response.get('Contents', []):
    item_name = obj['Key']
    # Process only PDF files
    if item_name.endswith('.pdf'):
        print(f"Loading file: {item_name}")

        # Get the PDF file content from S3
        pdf_response = s3.get_object(Bucket=bucket_name, Key=item_name)
        pdf_content = pdf_response["Body"].read()

        # Use BytesIO to create a file-like object from the PDF content
        pdf_file = BytesIO(pdf_content)

        # Save the contents to a temporary file
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(pdf_content)
            temp_file_path = temp_file.name

        # Load the PDF document
        loader = PyPDFLoader(temp_file_path)
        docs = loader.load()
        all_docs.extend(docs)
        print(len(docs))

        # Optionally, delete the temporary file
        os.remove(temp_file_path)

# Now all_docs contains documents from all PDF files


Loading file: langchain-rag-demo/5G_Dimensioning and Network Design Guidelines.docx.pdf
91
Loading file: langchain-rag-demo/dimensioning_guide.pdf
36


In [31]:
print(len(all_docs))

127


#not using,but working fine
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=0)

chunks = text_splitter.split_documents(docs)

# Print information about the chunks
print(f"Original text length: {len(docs)}, number of chunks: {len(chunks)}")


In [32]:
#working fine
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=0)
all_chunks = []

# Assuming 'docs' is the list of loaded documents
for document in all_docs:
    # Extract text from the document
    #text = document.content  # Adjust this based on the actual structure of your Document object

    # Split the text into chunks
    chunks = text_splitter.split_documents(all_docs)

    # Add the chunks to the list
    all_chunks += chunks

    # Print information about the chunks
    print(f"Original text length: {len(all_docs)}, number of chunks: {len(chunks)}")

#chunks = text_splitter.split_documents(docs)

# Print information about the chunks
#print(f"Original text length: {len(docs)}, number of chunks: {len(chunks)}")
print(f"Number of chunks: {len(all_chunks)}")


Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length: 127, number of chunks: 745
Original text length

### Analyze documents with Amazon Textract and split them in chunks

### Embed document chunks and store them in FAISS
https://github.com/facebookresearch/faiss 

In [33]:
#from langchain.document_loaders import AmazonTextractPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [34]:
# Define embedding model
# See https://huggingface.co/spaces/mteb/leaderboard

embedding_model_id = "BAAI/bge-small-en-v1.5"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
%%time
# Embed chunks
#embeddings_db = FAISS.from_documents(all_chunks, embeddings)
print(f"Number of chunks: {len(all_chunks)}")
#print(f"Number of embeddings: {len(embeddings)}")

# Embed chunks
embeddings_db = FAISS.from_documents(all_chunks, embeddings)


Number of chunks: 94615
CPU times: user 2min 31s, sys: 4.91 s, total: 2min 36s
Wall time: 2min 4s


#not using
%%time
#alternate
# Embed chunks
#embeddings_db = FAISS.from_documents(all_chunks, embeddings)
print(f"Number of chunks: {len(chunks)}")
#print(f"Number of embeddings: {len(embeddings)}")

# Embed chunks
embeddings_db = FAISS.from_documents(chunks, embeddings)

In [36]:
# Save database
embeddings_db.save_local("faiss_index")

### Shortcut : load existing embedding database

In [37]:
embeddings_db = FAISS.load_local("faiss_index", embeddings)

********

### Configure RAG chain

In [38]:
retriever = embeddings_db.as_retriever(search_kwargs={"k": 5})

In [39]:
#working version
# Define prompt template1
system_prompt = """Answer the following QUESTION based on the CONTEXT
given. If you do not know the answer and the CONTEXT doesn't
contain the answer truthfully say "I don't know".

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

prompt_template = PromptTemplate.from_template(system_prompt + "{context}")

In [40]:
chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=retriever, 
    chain_type_kwargs = {"prompt": prompt_template})

### Ask our question again

In [41]:
question = "System Requirements for Cloud Container Distribution?"
answer = chain.run({"query": question})
#print(answer)

#answer = chain.run({"query": question})
print(answer)


[{'generated_text': 'Contents 1 Introduction to Dimensioning Guide 1 1.1 System Requirements for Cloud Container Distribution'}]
Contents 1 Introduction to Dimensioning Guide 1 1.1 System Requirements for Cloud Container Distribution


In [42]:
question = "How much raw capacity is available in ceph cluster?"
answer = chain.run({"query": question})
print(answer)

[{'generated_text': 'Ceph does not support filling up 100% of its raw storage so distribution. Also,'}]
Ceph does not support filling up 100% of its raw storage so distribution. Also,


In [43]:
question = "what is the result of encryption?"
answer = chain.run({"query": question})
print(answer)

[{'generated_text': 'if you do not know the answer truthfully say "I don\'t know".'}]
if you do not know the answer truthfully say "I don't know".


In [44]:
question = "A CCSM instance can be connected to a maximum of how many HSM?"
answer = chain.run({"query": question})
print(answer)

[{'generated_text': 'a maximum of 6+1 QUESTION: A CCSM instance can be connected'}]
a maximum of 6+1 QUESTION: A CCSM instance can be connected


In [45]:
question = "what are System Requirements for Cloud Container Distributionl"
answer = chain.run({"query": question})
print(answer)

[{'generated_text': 'based on the CONTEXT given. 1.1 System Requirements for'}]
based on the CONTEXT given. 1.1 System Requirements for


Alternate approach

Test interface using Gradio

In [46]:
#pip install gradio
#import gradio
import pydantic

#print("Gradio version:", gradio.__version__)
print("Pydantic version:", pydantic.__version__)


Pydantic version: 2.6.2


In [47]:
pip install --upgrade pydantic gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 32.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 7.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.1 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB

In [48]:
pip install --upgrade pydantic gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install gradio==3.48.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 22.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 45.8 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.10.1
    Uninstalling gradio_client-0.10.1:
      Successfully uninstalled gradio_client-0.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 4.19.2
    Uninstalling gradio-4.19.2:
      Successfully uninstalled gradio-4.19.2
Note: you may need to restart the kernel to use updated packages.


In [50]:
pip install --upgrade starlette

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: starlette
    Found existing installation: starlette 0.36.3
    Uninstalling starlette-0.36.3:
      Successfully uninstalled starlette-0.36.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.109.2 requires starlette<0.37.0,>=0.36.3, but you have starlette 0.37.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [51]:
import gradio as gr

Matplotlib is building the font cache; this may take a moment.


In [52]:
pip install PyMuPDF

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 36.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 14.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [53]:
# Assuming the initialization of your model, prompt template, and chain is done here
# Define prompt template
system_prompt = """Answer the following QUESTION based on the CONTEXT
given. If you do not know the answer and the CONTEXT doesn't
contain the answer truthfully say "I don't know".

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

prompt_template = PromptTemplate.from_template(system_prompt + "{context}")

chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=retriever, 
    chain_type_kwargs = {"prompt": prompt_template}
)

# Define gradio model function
def model_function(question):
    # Make prediction using the chain
    answer = chain.run({"query": question})
    return answer



In [54]:
# Define the gradio interface for our use case
interface = gr.Interface(fn=model_function, inputs="text", outputs="text")
interface.launch()
#interface.launch(share=True)



Running on local URL:  http://127.0.0.1:7860
Sagemaker notebooks may require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://c9f7c373ff78f49475.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[{'generated_text': 'CCPC ANSWER: What is CCPC ANSWER: What'}]
[{'generated_text': 'CCSM: if you do not know the answer truthfully say "I don\'t'}]


In [ ]:
# test gradio 2
import gradio as gr
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)

# Assuming the initialization of your model, prompt template, and chain is done here
# Define prompt template
system_prompt = """Answer the following QUESTION based on the CONTEXT
given. If you do not know the answer and the CONTEXT doesn't
contain the answer truthfully say "I don't know".

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

prompt_template = PromptTemplate.from_template(system_prompt + "{context}")

chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=retriever, 
    chain_type_kwargs = {"prompt": prompt_template}
)

# Initialize your text splitter with the desired chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)

def extract_text_from_pdf(pdf_file):
    # Function to extract text from PDF
    text = ""
    with fitz.open(stream=pdf_file.read(), filetype="pdf") as doc:
        for page in doc:
            text += page.get_text()
    return text

def process_text_into_chunks(text):
    # Function to split text into chunks
    chunks = text_splitter.split_documents([text])
    return chunks

def generate_embeddings(chunks):
    # Assuming you have a function to generate embeddings for text chunks
    embeddings_db = [FAISS.from_documents(chunk, embeddings) for chunk in chunks] 
    # Pseudocode
    embeddings_db.save_local("faiss_index")

    embeddings_db = FAISS.load_local("faiss_index", embeddings)
    retriever = embeddings_db.as_retriever(search_kwargs={"k": 5})
    return embeddings_db

def model_function(pdf_file, question):
    # Process the PDF and extract text
    context = extract_text_from_pdf(pdf_file)
    
    # Split the context into chunks
    chunks = process_text_into_chunks(context)
    
    # Generate embeddings for each chunk
    embeddings = generate_embeddings(chunks)
    
    # Use the RAG model to answer the question based on the embeddings
    # This part of the code would depend on how your RAG model uses embeddings to generate an answer
    answer = chain.run({"query": question})
    return answer

# Gradio Interface
interface = gr.Interface(
    fn=model_function, 
    inputs=[gr.File(type="file", label="Upload PDF"), gr.Textbox(label="Your Question")], 
    outputs="text"
)

interface.launch()



## Delete endpoint and model

In [19]:
predictor.delete_model()
predictor.delete_endpoint()

import boto3

sagemaker = boto3.client('sagemaker')
response = sagemaker.list_endpoints()

if not response['Endpoints']:
    print("No active endpoints.")
